In [1]:
import os
import math
import docx
from bitarray import bitarray

In [82]:
# 读入文字
# file=docx.Document("./Introduction to Data Compression.docx")

In [83]:
# origin_data = ""
# for i,para in enumerate(file.paragraphs):
#     origin_data += para.text
#     origin_data += '\n'

In [5]:
# 0-9 48-57
# a-z 97-122
# A-Z 65-90

In [6]:
# 改进
# 大小写 压缩
# 给出一个标志位

In [251]:
# 计算大小
# plogp
ff = open('./inputs.txt',encoding='utf-8')
origin_data = ""
for line in ff.readlines():
    origin_data+=line
def cal(data):
    sums = 0
    total = len(data)
    count = {}
    for i in data:
        if i not in count.keys():
            count[i] = 0
        count[i]+=1
    for key in count:
        temp = count[key]/total
        sums -= temp*math.log(temp)
    return count,sums

In [252]:
count,h = cal(origin_data)

In [253]:
h

3.026150844320891

# 编码

In [62]:
# 16个bit编码下标 8bit编码ascii
def get_output(origin):
    output_buffer = bitarray(endian='big')
    word = b''
    group = {}
    group[b''] = [0,0,b'']
    ct = 1
    i = 0
    while i<len(origin):
        chars = origin[i:i+1]
        word += chars
        if word not in group.keys():
            if len(word)==1:
                group[chars] = [0,ct,chars]
                output_buffer.frombytes(bytes([0,0]))
                output_buffer.frombytes(bytes(chars))
            else:
                tmp = group[word[:-1]][1]
                output_buffer.frombytes(bytes([tmp>>8,tmp&0xff]))
                group[word] = [tmp,ct,chars]
                output_buffer.frombytes(bytes(chars))
            ct += 1
            word=b''
        i+=1
        # 最后几个字符
        if i==len(origin):
            if word in group.keys():
                tmp = group[word][1]
                output_buffer.frombytes(bytes([tmp>>8,tmp&0xff]))
                output_buffer.frombytes(b'')
    return group, output_buffer

In [63]:
with open('./inputs.txt', 'rb') as input_file:
    origin = input_file.read()

In [64]:
gps, out_buff = get_output(origin)

In [254]:
gps

{b'': [0, 0, b''],
 b'\xef': [0, 1, b'\xef'],
 b'\xbb': [0, 2, b'\xbb'],
 b'\xbf': [0, 3, b'\xbf'],
 b'I': [0, 4, b'I'],
 b'n': [0, 5, b'n'],
 b't': [0, 6, b't'],
 b'r': [0, 7, b'r'],
 b'o': [0, 8, b'o'],
 b'd': [0, 9, b'd'],
 b'u': [0, 10, b'u'],
 b'c': [0, 11, b'c'],
 b'ti': [6, 12, b'i'],
 b'on': [8, 13, b'n'],
 b' ': [0, 14, b' '],
 b'to': [6, 15, b'o'],
 b' d': [14, 16, b'd'],
 b'a': [0, 17, b'a'],
 b'ta': [6, 18, b'a'],
 b' c': [14, 19, b'c'],
 b'om': [8, 20, b'm'],
 b'p': [0, 21, b'p'],
 b're': [7, 22, b'e'],
 b's': [0, 23, b's'],
 b'si': [23, 24, b'i'],
 b'on\r': [13, 25, b'\r'],
 b'\n': [0, 26, b'\n'],
 b'\r': [0, 27, b'\r'],
 b'\nI': [26, 28, b'I'],
 b'n ': [5, 29, b' '],
 b'th': [6, 30, b'h'],
 b'e': [0, 31, b'e'],
 b' l': [14, 32, b'l'],
 b'as': [17, 33, b's'],
 b't ': [6, 34, b' '],
 b'de': [9, 35, b'e'],
 b'ca': [11, 36, b'a'],
 b'de,': [35, 37, b','],
 b' w': [14, 38, b'w'],
 b'e ': [31, 39, b' '],
 b'h': [0, 40, b'h'],
 b'av': [17, 41, b'v'],
 b'e b': [39, 42, b'b'],
 b

In [249]:
len(origin)

14453

In [247]:
len(out_buff)

87736

In [250]:
87736/14453

6.070435203763925

In [65]:
with open('./out', 'wb') as output_file:
    output_file.write(out_buff.tobytes())

# 解码

In [66]:
outdata = bitarray(endian='big')
with open('./out', 'rb') as input_file:
    outdata.fromfile(input_file)

In [67]:
def decode(origin):
    word = b''
    group = {}
    group[0] = b''
    ct = 1
    i = 0
    output_buffer = []
    while i<len(outdata):
#         ot8 = outdata[i:i+8].tobytes()
#         ot16 = outdata[i+8:i+16].tobytes()
        # !!!!!!!!!! <<运算优先级低ddddddddddd
        index = (ord(outdata[i:i+8].tobytes())<<8) + (ord(outdata[i+8:i+16].tobytes()))
        char = outdata[i+16:i+24].tobytes()
        word = group[index] + char
        output_buffer.append(word)
        group[ct] = word
        ct+=1
        i+=24
    return group,output_buffer

In [69]:
g, outbufu = decode(outdata)

In [70]:
out_data =  b''.join(outbufu)

In [71]:
with open('decode.txt', 'wb') as output_file:
    output_file.write(out_data)

# Improve

In [214]:
with open('./inputs.txt', 'rb') as input_file:
    origin = input_file.read()
def improve_encode(origin):
    # 预存ascii表
    p = b''
    output_buffer = bitarray(endian='big')
    table = {}
    # 存成byte
    for i in range(256):
        table[(i).to_bytes(1,'big')] = i
    k = 0
    index = 256
    while k<len(origin):
        c = origin[k:k+1]
        if p+c not in table.keys():
            tmp = table[p]
            # 16bit
            output_buffer.frombytes(bytes([tmp>>8,tmp&0xff]))
            table[p+c] = index
            index+=1
            p = c
        else:
            p = p+c
        k+=1
        if k>=len(outdata):
            tmp = table[p]
            output_buffer.frombytes(bytes([tmp>>8,tmp&0xff]))
    return table,output_buffer

In [215]:
table,improveout = improve_encode(origin)

In [255]:
len(improveout)

75120

In [257]:
75120/14453

5.19753684356189

In [256]:
table

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [216]:
with open('./improveout', 'wb') as output_file:
    output_file.write(improveout.tobytes())

# Improve 解码

In [241]:
outdata = bitarray(endian='big')
with open('./improveout', 'rb') as input_file:
    outdata.fromfile(input_file)
def decode(origin):
    # 预存ascii表
    output_buffer = []
    table = {}
    # 存成byte
    for i in range(256):
        table[i] = (i).to_bytes(1,'big')
    # 初始化
    num = (ord(outdata[0:0+8].tobytes())<<8) + (ord(outdata[0+8:0+16].tobytes()))
    c = table[num]
    output_buffer.append(c)
    # 解码需要还原出编码时的用的字典
    p = num
    i = 16
    index = 256
    while i<len(outdata):
        num = (ord(outdata[i:i+8].tobytes())<<8) + (ord(outdata[i+8:i+16].tobytes()))
        if num in table:
            addc = table[num]
            c = addc[0]
            # 转化为bytes b'...'
            c = (c).to_bytes(1,'big')
            p_c =  table[p] + c
            table[index] = p_c
            index+=1
            output_buffer.append(addc)
            p = num
        else:
            c = table[p][0]
            c = (c).to_bytes(1,'big')
            p_c =  table[p] + c
            p = num
            table[num] = p_c
            index+=1
            output_buffer.append(p_c)
        i+=16
#         if i>=len(outdata):
#             output_buffer.append(table[num])
    return table,output_buffer

In [242]:
tb,out_buffer = decode(outdata)

In [243]:
out_data =  b''.join(out_buffer)

In [244]:
with open('imporvedecode.txt', 'wb') as output_file:
    output_file.write(out_data)

In [144]:
# def get_table(origin_data):
#     encodechar = set()
#     group = {}
#     group[""] = [0,0,""]
#     word = ""
#     ct = 1
#     for i,chars in enumerate(origin_data):
#         word += chars
#         if word not in group.keys():
#             if len(word)==1:
#                 group[chars] = [0,ct,chars]
#                 encodechar.add(chars)
#             else:
#                 group[word] = [group[word[:-1]][1],ct,chars]
#                 encodechar.add(chars)
#             ct += 1
#             word=""
#     return group,encodechar

In [57]:
# # {字符组:[前面的下标，自己的下标 ,要编码的字符]}
# # 先分割，统计词数
# def get_table(origin_data):
#     group = {}
#     word = ""
#     ct = 1
#     for i,chars in enumerate(origin_data):
#         word += chars
#         if word not in group.keys():
#             if len(word)==1:
#                 group[chars] = [0,ct,chars]
#             else:
#                 group[word] = [group[word[:-1]][1],ct,chars]
#             ct += 1
#             word=""
# def get_output(origin_data):
#     output = ""
#     word = ""
#     group = {}
#     group[""] = [0,0,""]
#     ct = 1
#     for i,chars in enumerate(origin_data):
# #         if chars==" ":
# #             output+=chars
# #             output+=str(group[word][1])
#         if chars in '0123456789':
#             # 遇到数字
#             output+=str(group[word][1])+"-1"+chars
#             word=""
#         else:
#             word += chars
#             if word not in group.keys():
#                 if len(word)==1:
#                     group[chars] = [0,ct,chars]
#                     output+="0"+chars
#                 else:
#                     tmp = group[word[:-1]][1]
#                     group[word] = [tmp,ct,chars]
#                     output+=str(tmp)+chars
#                 ct += 1
#                 word=""
#     return group, output

In [58]:
# gp, out= get_output(origin_data)

In [19]:
# f = open('out.txt','a',encoding='utf-8')
# f.write(out)
# f.close()

In [19]:
# ap = []
# ccs = []
# decodeout = ""
# group = {}
# group["0"] = ""
# word = ""
# ct = 1
# num = ""
# index = 0
# prex = ""
# while index<len(out):
#     char = out[index]
#     # 注意 0-1这种情况 -在前面没出现过
#     if prex=="" and char=="-" and out[index+1]=="1":
#         index +=2
#         char = out[index]
#         decodeout += char
#     else:
#         if char in "0123456789":
#             prex+=char
#         else:
#             ap.append(prex)
#             ccs.append(char)
#             temp = group[prex] + char
#             group[str(ct)] = temp
#             ct+=1
#             decodeout += temp
#             prex = ""
#     index+=1

In [ ]:
# gps,enc = get_table(origin_data)
# # 决定下标的bit数
# ind_bit = 12
# def get_output(origin_data):
#     output = ""
#     word = ""
#     group = {}
#     group[""] = [0,0,""]
#     ct = 1
#     for i,chars in enumerate(origin_data):
#             word += chars
#             if word not in group.keys():
#                 if len(word)==1:
#                     group[chars] = [0,ct,chars]
#                     output+="0"+chars
#                 else:
#                     tmp = group[word[:-1]][1]
#                     group[word] = [tmp,ct,chars]
#                     output+=str(tmp)+chars
#                 ct += 1
#                 word=""
#     return group, output